# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [92]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [76]:
#count schools
total_schools=school_data_complete["school_name"].nunique()
#count students
total_students=school_data_complete["Student ID"].count()
# go back to the school table to get the budget
budget=school_data["budget"].sum()
#calculate math scores
avg_math=school_data_complete["math_score"].mean()
#calculate reading scores
avg_read=school_data_complete["reading_score"].mean()
#count students who passed math
passed_math=school_data_complete["Student ID"].loc[school_data_complete["math_score"] >= 70].count()
#calculate percentages
mpassed_pct=passed_math/total_students * 100
#count students who passed reading
passed_read=school_data_complete["Student ID"].loc[school_data_complete["reading_score"] >= 70].count()
#calculate percentages
rpassed_pct=passed_read/total_students * 100
# do the overall percentages of who passed
opassed_pct=(passed_read+passed_math)/2/total_students*100
# well I did not read this clearly, but thought overall meant passed both reading and math.
#passed_over=school_data_complete["Student ID"].loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"]>=70)].count()
#opassed_pct=passed_over/total_students * 100
# note the percentage of studings passing both is 65.17

#make the summary dataframe
summary_df=pd.DataFrame({"Total Schools": [total_schools],
                         "Total Students": [total_students],
                         "Total Budget": [budget],
                         "Average Math Score": [avg_math],
                         "Average Reading Score" : [avg_read],
                         "% Passing Math":[mpassed_pct],
                         "% Passing Reading":[rpassed_pct],
                         "% Overall Passing Rate":[opassed_pct]
                    })
#clean it up
summary_df["Total Students"]=summary_df["Total Students"].map( "{:,.0f}".format)
summary_df["Total Budget"]=summary_df["Total Budget"].map( "${:,.2f}".format)
summary_df["Average Math Score"]=summary_df["Average Math Score"].map( "{:.2f}".format)
summary_df["Average Reading Score"]=summary_df["Average Reading Score"].map( "{:.2f}".format)
summary_df["% Passing Math"]=summary_df["% Passing Math"].map( "{:.2f} %".format)
summary_df["% Passing Reading"]=summary_df["% Passing Reading"].map( "{:.2f} %".format)
summary_df["% Overall Passing Rate"]=summary_df["% Overall Passing Rate"].map( "{:.2f} %".format)

#display the frame
summary_df



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98 %,85.81 %,80.39 %


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [127]:
#create groupby
grouped_schools=school_data_complete.groupby(["school_name","type"])
#make a data frame
top_performers=grouped_schools.count()
#start building columns
top_performers["Total Students"]=top_performers["Student ID"]
#get the whole school budget as it's listed in the student, I only need one value, so I chose max
school_budget=grouped_schools.max()
#add it to the df
top_performers["Total School Budget"]=school_budget["budget"]
#drop all the other columns
top_performers=top_performers[["Total Students","Total School Budget"]]
#add per student column
top_performers["Per Student Budget"]=top_performers["Total School Budget"]/top_performers["Total Students"]
#get the average df
avg_scores=grouped_schools.mean()
#add those to the summary
top_performers["Average Math Score"]=avg_scores["math_score"]
top_performers["Average Reading Score"]=avg_scores["reading_score"]
#make a new df that contains only students who passed math
math_passers=school_data_complete[["school_name","Student ID","math_score"]].loc[school_data_complete["math_score"] >= 70]
#group them by school and count them
grouped_mp=math_passers.groupby("school_name").count()
#add them to the column at the end
top_performers["% Passing Math"]=grouped_mp["Student ID"]/top_performers["Total Students"]*100
#make a new df that contains only students who passed reading
read_passers=school_data_complete[["school_name","Student ID","reading_score"]].loc[school_data_complete["reading_score"] >= 70]
#group them by school and count them
grouped_rp=read_passers.groupby("school_name").count()
#add them to the df in a new column
top_performers["% Passing Reading"]=grouped_rp["Student ID"]/top_performers["Total Students"]*100
#do the overall calculations
top_performers["% Overall Passing Rate"]=(top_performers["% Passing Reading"]+top_performers["% Passing Math"])/2
# reset the index for type
top_performers=top_performers.reset_index(level="type")
#rename the type
top_performers=top_performers.rename(columns={"type": "School Type"})
#get rid of the schoool_name at the top
top_performers.index.names = ['']
#copying this for next one so I don't have to deal with string conversion after I add the formatting
bottom_performers=top_performers.copy()
#sort the thing
top_performers=top_performers.sort_values(by=["% Overall Passing Rate"], ascending=False)
#format the output
top_performers["Total Students"]=top_performers["Total Students"].map( "{:,.0f}".format)
top_performers["Total School Budget"]=top_performers["Total School Budget"].map( "${:,.2f}".format)
top_performers["Per Student Budget"]=top_performers["Per Student Budget"].map( "${:,.2f}".format)
top_performers["Average Math Score"]=top_performers["Average Math Score"].map( "{:.2f}".format)
top_performers["Average Reading Score"]=top_performers["Average Reading Score"].map( "{:.2f}".format)
top_performers["% Passing Math"]=top_performers["% Passing Math"].map( "{:.2f} %".format)
top_performers["% Passing Reading"]=top_performers["% Passing Reading"].map( "{:.2f} %".format)
top_performers["% Overall Passing Rate"]=top_performers["% Overall Passing Rate"].map( "{:.2f} %".format)

top_performers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13 %,97.04 %,95.59 %
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27 %,97.31 %,95.29 %
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59 %,95.95 %,95.27 %
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39 %,97.14 %,95.27 %
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87 %,96.54 %,95.20 %


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [128]:
#This starts with the bottom performers that was copied from the last one
bottom_performers=bottom_performers.sort_values(by=["% Overall Passing Rate"], ascending=True)
#format the output
bottom_performers["Total Students"]=bottom_performers["Total Students"].map( "{:,.0f}".format)
bottom_performers["Total School Budget"]=bottom_performers["Total School Budget"].map( "${:,.2f}".format)
bottom_performers["Per Student Budget"]=bottom_performers["Per Student Budget"].map( "${:,.2f}".format)
bottom_performers["Average Math Score"]=bottom_performers["Average Math Score"].map( "{:.2f}".format)
bottom_performers["Average Reading Score"]=bottom_performers["Average Reading Score"].map( "{:.2f}".format)
bottom_performers["% Passing Math"]=bottom_performers["% Passing Math"].map( "{:.2f} %".format)
bottom_performers["% Passing Reading"]=bottom_performers["% Passing Reading"].map( "{:.2f} %".format)
bottom_performers["% Overall Passing Rate"]=bottom_performers["% Overall Passing Rate"].map( "{:.2f} %".format)
bottom_performers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37 %,80.22 %,73.29 %
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99 %,80.74 %,73.36 %
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68 %,81.32 %,73.50 %
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06 %,81.22 %,73.64 %
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31 %,79.30 %,73.80 %


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [91]:
school_data_complete.head()
grade_math=school_data[["school_name","grade","math_score"]]
grade_math["9th"]=

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
